In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        )

    def forward(self, x):
        return x + self.block(x)

# Lightweight CNN-based Super-Resolution Model with Upsample
class SuperResolutionCNN(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_channels=64, num_blocks=5, scale_factor=1):
        super(SuperResolutionCNN, self).__init__()

        # Initial feature extraction
        self.entry = nn.Conv2d(in_channels, num_channels, kernel_size=3, padding=1)

        # Residual Blocks
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(num_channels) for _ in range(num_blocks)]
        )

        # Feature aggregation
        self.exit = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)

        # Upsampling Layer (PixelShuffle)
        if scale_factor == 1:
            self.upsample = nn.Identity()
        else:
            self.upsample = nn.Sequential(
                nn.Conv2d(num_channels, num_channels * (scale_factor ** 2), kernel_size=3, padding=1),
                nn.PixelShuffle(scale_factor),
                nn.ReLU(inplace=True)
            )


        # Final output layer
        self.output = nn.Conv2d(num_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x, return_features=False):
        feat = self.entry(x)
        intermediate_feats = []
        res = feat
        for i, block in enumerate(self.res_blocks):
            res = block(res)
            print(f"Block {i} output shape: {res.shape}, NaN detected: {torch.isnan(res).any().item()}")
            intermediate_feats.append(res)

        res = self.exit(res)
        print(f"After exit conv shape: {res.shape}, NaN detected: {torch.isnan(res).any().item()}")
           
        up = self.upsample(res + 0.1 * feat)
        print(f"After upsample shape: {up.shape}, NaN detected: {torch.isnan(up).any().item()}")
        out = self.output(up)
        print(f"Output shape: {out.shape}, NaN detected: {torch.isnan(out).any().item()}")

        if return_features:
            return out, intermediate_feats
        else:
            return out

In [9]:
model = SuperResolutionCNN()
print(model)

SuperResolutionCNN(
  (entry): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (res_blocks): Sequential(
    (0): ResidualBlock(
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (1): ResidualBlock(
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (2): ResidualBlock(
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (3): ResidualBlock(
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import math
from tqdm import tqdm

In [11]:
import random

class PatchDataset(Dataset):
    def __init__(self, lr_dir, hr_dir, transform=None, indices=None):
        self.lr_files = sorted(os.listdir(lr_dir))
        self.hr_files = sorted(os.listdir(hr_dir))
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir
        self.transform = transform

        if indices is None:
            self.indices = list(range(len(self.lr_files)))
        else:
            self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        lr = Image.open(os.path.join(self.lr_dir, self.lr_files[real_idx])).convert('RGB')
        hr = Image.open(os.path.join(self.hr_dir, self.hr_files[real_idx])).convert('RGB')

        if self.transform:
            lr = self.transform(lr)
            hr = self.transform(hr)

        return lr, hr


In [12]:
def psnr(pred, target):
    mse = torch.mean((pred - target) ** 2)
    if mse == 0:
        return 100
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

In [13]:
from skimage.metrics import structural_similarity as ssim
import numpy as np

def ssim(pred, target):
    batch_size = pred.size(0)
    ssim_total = 0.0

    for i in range(batch_size):
        pred_img = pred[i].permute(1, 2, 0).cpu().numpy()
        target_img = target[i].permute(1, 2, 0).cpu().numpy()

        ssim_score = compare_ssim(pred_img, target_img, multichannel=True, data_range=1.0)
        ssim_total += ssim_score

    return ssim_total / batch_size

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = ToTensor()
full_size = len(os.listdir('../data/patches/LR_patches1'))
print(full_size)
# Randomly sample 500 unique indices
sample_indices = random.sample(range(full_size), 1000)
print(len(sample_indices))

# Create dataset with sampled indices
sampled_dataset = PatchDataset('../data/patches/LR_patches1', '../data/patches/HR_patches', transform=transform, indices=sample_indices)

55510
1000


In [15]:
train_size = int(0.8 * 1000)  # 80% train
test_size = 1000 - train_size

train_indices = sample_indices[:train_size]
test_indices = sample_indices[train_size:]

train_dataset = PatchDataset('../data/patches/LR_patches1', '../data/patches/HR_patches', transform=transform, indices=train_indices)
test_dataset = PatchDataset('../data/patches/LR_patches1', '../data/patches/HR_patches', transform=transform, indices=test_indices)

In [16]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [17]:
model = SuperResolutionCNN().to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for lr, hr in tqdm(train_loader):
        lr, hr = lr.to(device), hr.to(device)

        optimizer.zero_grad()
        sr = model(lr)
        loss = criterion(sr, hr)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

  0%|          | 0/100 [00:00<?, ?it/s]

Block 0 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 1 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 2 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 3 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 4 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After exit conv shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After upsample shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Output shape: torch.Size([8, 3, 192, 192]), NaN detected: False


  1%|          | 1/100 [00:01<01:53,  1.15s/it]

Block 0 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 1 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 2 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 3 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 4 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After exit conv shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After upsample shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Output shape: torch.Size([8, 3, 192, 192]), NaN detected: False


  2%|▏         | 2/100 [00:01<01:11,  1.36it/s]

...

Block 0 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 1 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 2 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 3 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 4 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After exit conv shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After upsample shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Output shape: torch.Size([8, 3, 192, 192]), NaN detected: False


 99%|█████████▉| 99/100 [01:23<00:00,  1.19it/s]

Block 0 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 1 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 2 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 3 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 4 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After exit conv shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After upsample shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Output shape: torch.Size([8, 3, 192, 192]), NaN detected: False


100%|██████████| 100/100 [01:24<00:00,  1.18it/s]

Epoch [15/15], Loss: 0.0173


In [18]:
import torch
from skimage.metrics import structural_similarity as compare_ssim

from skimage.metrics import structural_similarity as compare_ssim

def ssim(pred, target):
    batch_size = pred.size(0)
    ssim_total = 0.0
    for i in range(batch_size):
        pred_img = pred[i].permute(1, 2, 0).cpu().numpy()
        target_img = target[i].permute(1, 2, 0).cpu().numpy()
        
        ssim_score = compare_ssim(pred_img, target_img, multichannel=True, data_range=1.0, win_size=7, channel_axis=2)
        ssim_total += ssim_score
    return ssim_total / batch_size

    
def psnr(pred, target):
    mse = torch.mean((pred - target) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

model.eval()
total_psnr = 0.0
total_ssim = 0.0

with torch.no_grad():
    for lr, hr in test_loader:
        lr, hr = lr.to(device), hr.to(device)
        sr = model(lr)
        total_psnr += psnr(sr, hr).item()
        total_ssim += ssim(sr, hr)  # <-- uncommented this line

avg_psnr = total_psnr / len(test_loader)
avg_ssim = total_ssim / len(test_loader)

print(f"Average PSNR on test set: {avg_psnr:.2f} dB")
print(f"Average SSIM on test set: {avg_ssim:.4f}")


Block 0 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 1 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 2 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 3 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 4 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After exit conv shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After upsample shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Output shape: torch.Size([8, 3, 192, 192]), NaN detected: False
Block 0 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 1 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 2 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 3 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
Block 4 output shape: torch.Size([8, 64, 192, 192]), NaN detected: False
After exit conv shape: torch.Size([8, 64, 192, 192]), NaN d

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')